In [40]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

In [ ]:
# 1. 加载数据
df = pd.read_csv('Results_21MAR2022_nokcaladjust.csv')
df.head()

,mc_run_id,grouping,mean_ghgs,mean_land,mean_watscar,mean_eut,mean_ghgs_ch4,mean_ghgs_n2o,mean_bio,mean_watuse,...,sd_eut,sd_ghgs_ch4,sd_ghgs_n2o,sd_bio,sd_watuse,sd_acid,n_participants,sex,diet_group,age_group
0,1,fish_female_20-29,5.377071,8.589807,14012.27478,23.553012,1.165277,0.344528,478.802638,669.154382,...,13.788241,0.913103,0.238819,400.289973,402.372408,13.774998,1329,female,fish,20-29
1,1,fish_female_30-39,5.376457,8.620238,15030.26610,23.831419,1.152422,0.326276,469.573862,697.161548,...,13.466037,0.845564,0.210539,375.301226,420.147350,13.077207,2024,female,fish,30-39
2,1,fish_female_40-49,5.169838,8.304831,15441.25934,23.419815,1.095640,0.305898,435.534050,687.733569,...,12.692978,0.773937,0.190561,333.613445,404.151916,12.959012,1689,female,fish,40-49
3,1,fish_female_50-59,4.933744,7.880759,16244.68021,23.459450,0.995247,0.285800,382.097727,686.763592,...,13.825273,0.669932,0.166206,271.061989,429.965087,13.155250,946,female,fish,50-59
4,1,fish_female_60-69,5.428442,8.701867,18364.95302,25.469176,1.127873,0.315651,435.578230,751.050809,...,14.538055,0.907853,0.196968,396.811097,471.376883,14.177048,492,female,fish,60-69


In [13]:
# 检查数据基础信息
print(f"缺失值统计:\n{df.isnull().sum()}")

缺失值统计:
mc_run_id         0
grouping          0
mean_ghgs         0
mean_land         0
mean_watscar      0
mean_eut          0
mean_ghgs_ch4     0
mean_ghgs_n2o     0
mean_bio          0
mean_watuse       0
mean_acid         0
sd_ghgs           0
sd_land           0
sd_watscar        0
sd_eut            0
sd_ghgs_ch4       0
sd_ghgs_n2o       0
sd_bio            0
sd_watuse         0
sd_acid           0
n_participants    0
sex               0
diet_group        0
age_group         0
dtype: int64


In [21]:
env_indicators = ['ghgs', 'land', 'watscar', 'eut', 'ghgs_ch4', 'ghgs_n2o', 'bio', 'watuse', 'acid']
mean_cols = [f'mean_{base}' for base in env_indicators]
sd_cols = [f'sd_{base}' for base in env_indicators]

In [22]:
calu_df = df.copy()
calu_df.head()

,mc_run_id,grouping,mean_ghgs,mean_land,mean_watscar,mean_eut,mean_ghgs_ch4,mean_ghgs_n2o,mean_bio,mean_watuse,...,sd_eut,sd_ghgs_ch4,sd_ghgs_n2o,sd_bio,sd_watuse,sd_acid,n_participants,sex,diet_group,age_group
0,1,fish_female_20-29,5.377071,8.589807,14012.27478,23.553012,1.165277,0.344528,478.802638,669.154382,...,13.788241,0.913103,0.238819,400.289973,402.372408,13.774998,1329,female,fish,20-29
1,1,fish_female_30-39,5.376457,8.620238,15030.26610,23.831419,1.152422,0.326276,469.573862,697.161548,...,13.466037,0.845564,0.210539,375.301226,420.147350,13.077207,2024,female,fish,30-39
2,1,fish_female_40-49,5.169838,8.304831,15441.25934,23.419815,1.095640,0.305898,435.534050,687.733569,...,12.692978,0.773937,0.190561,333.613445,404.151916,12.959012,1689,female,fish,40-49
3,1,fish_female_50-59,4.933744,7.880759,16244.68021,23.459450,0.995247,0.285800,382.097727,686.763592,...,13.825273,0.669932,0.166206,271.061989,429.965087,13.155250,946,female,fish,50-59
4,1,fish_female_60-69,5.428442,8.701867,18364.95302,25.469176,1.127873,0.315651,435.578230,751.050809,...,14.538055,0.907853,0.196968,396.811097,471.376883,14.177048,492,female,fish,60-69


In [23]:
# 3. 计算变异系数(CV)
for base in env_indicators:
    mean_col = f'mean_{base}'
    sd_col = f'sd_{base}'
    if mean_col in calu_df.columns and sd_col in calu_df.columns:
        calu_df[f'cv_{base}'] = calu_df[sd_col] / calu_df[mean_col]
calu_df.head()

,mc_run_id,grouping,mean_ghgs,mean_land,mean_watscar,mean_eut,mean_ghgs_ch4,mean_ghgs_n2o,mean_bio,mean_watuse,...,age_group,cv_ghgs,cv_land,cv_watscar,cv_eut,cv_ghgs_ch4,cv_ghgs_n2o,cv_bio,cv_watuse,cv_acid
0,1,fish_female_20-29,5.377071,8.589807,14012.27478,23.553012,1.165277,0.344528,478.802638,669.154382,...,20-29,0.634062,0.694508,0.628880,0.585413,0.783593,0.693178,0.836023,0.601315,0.596724
1,1,fish_female_30-39,5.376457,8.620238,15030.26610,23.831419,1.152422,0.326276,469.573862,697.161548,...,30-39,0.587847,0.631104,0.605818,0.565054,0.733728,0.645279,0.799238,0.602654,0.551064
2,1,fish_female_40-49,5.169838,8.304831,15441.25934,23.419815,1.095640,0.305898,435.534050,687.733569,...,40-49,0.561257,0.587256,0.591992,0.541976,0.706379,0.622955,0.765987,0.587658,0.542589
3,1,fish_female_50-59,4.933744,7.880759,16244.68021,23.459450,0.995247,0.285800,382.097727,686.763592,...,50-59,0.556976,0.591723,0.655199,0.589326,0.673132,0.581545,0.709405,0.626074,0.554468
4,1,fish_female_60-69,5.428442,8.701867,18364.95302,25.469176,1.127873,0.315651,435.578230,751.050809,...,60-69,0.615610,0.667712,0.627117,0.570810,0.804925,0.624005,0.910998,0.627623,0.545427


In [26]:
# 4. 统一量纲 - 归一化mean_指标
data_to_normalize = calu_df[mean_cols]
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(data_to_normalize)

In [27]:
# 将归一化后的数据添加回DataFrame
for i, base in enumerate(env_indicators):
    calu_df[f'norm_{base}'] = normalized_data[:, i]

In [28]:
calu_df.head()

,mc_run_id,grouping,mean_ghgs,mean_land,mean_watscar,mean_eut,mean_ghgs_ch4,mean_ghgs_n2o,mean_bio,mean_watuse,...,cv_acid,norm_ghgs,norm_land,norm_watscar,norm_eut,norm_ghgs_ch4,norm_ghgs_n2o,norm_bio,norm_watuse,norm_acid
0,1,fish_female_20-29,5.377071,8.589807,14012.27478,23.553012,1.165277,0.344528,478.802638,669.154382,...,0.596724,0.146738,0.063621,0.123640,0.153460,0.150321,0.119074,0.226080,0.129890,0.168919
1,1,fish_female_30-39,5.376457,8.620238,15030.26610,23.831419,1.152422,0.326276,469.573862,697.161548,...,0.551064,0.146714,0.063949,0.137388,0.155965,0.148559,0.109788,0.221237,0.137146,0.175497
2,1,fish_female_40-49,5.169838,8.304831,15441.25934,23.419815,1.095640,0.305898,435.534050,687.733569,...,0.542589,0.138615,0.060551,0.142938,0.152262,0.140779,0.099422,0.203374,0.134703,0.177053
3,1,fish_female_50-59,4.933744,7.880759,16244.68021,23.459450,0.995247,0.285800,382.097727,686.763592,...,0.554468,0.129360,0.055982,0.153788,0.152618,0.127022,0.089198,0.175331,0.134452,0.175447
4,1,fish_female_60-69,5.428442,8.701867,18364.95302,25.469176,1.127873,0.315651,435.578230,751.050809,...,0.545427,0.148752,0.064828,0.182421,0.170700,0.145195,0.104383,0.203397,0.151108,0.198514


In [29]:
# 基于变异系数创建可靠性权重
for base in env_indicators:
    cv_col = f'cv_{base}'
    if cv_col in calu_df.columns:
        # 计算可靠性权重 (CV越小，权重越大)
        calu_df[f'reliability_{base}'] = 1 / (1 + calu_df[cv_col])
        
        # 应用权重创建加权指标
        norm_col = f'norm_{base}'
        calu_df[f'weighted_{base}'] = calu_df[norm_col] * calu_df[f'reliability_{base}']

# 创建综合环境影响指标
weighted_indicator_cols = [f'weighted_{base}' for base in env_indicators]
calu_df['total_env_impact'] = calu_df[weighted_indicator_cols].mean(axis=1)
calu_df.head()

,mc_run_id,grouping,mean_ghgs,mean_land,mean_watscar,mean_eut,mean_ghgs_ch4,mean_ghgs_n2o,mean_bio,mean_watuse,...,weighted_ghgs_ch4,reliability_ghgs_n2o,weighted_ghgs_n2o,reliability_bio,weighted_bio,reliability_watuse,weighted_watuse,reliability_acid,weighted_acid,total_env_impact
0,1,fish_female_20-29,5.377071,8.589807,14012.27478,23.553012,1.165277,0.344528,478.802638,669.154382,...,0.084280,0.590605,0.070325,0.544656,0.123136,0.624487,0.081114,0.626282,0.105791,0.084966
1,1,fish_female_30-39,5.376457,8.620238,15030.26610,23.831419,1.152422,0.326276,469.573862,697.161548,...,0.085688,0.607800,0.066729,0.555791,0.122961,0.623965,0.085574,0.644719,0.113147,0.087879
2,1,fish_female_40-49,5.169838,8.304831,15441.25934,23.419815,1.095640,0.305898,435.534050,687.733569,...,0.082501,0.616160,0.061260,0.566256,0.115161,0.629859,0.084844,0.648261,0.114776,0.086001
3,1,fish_female_50-59,4.933744,7.880759,16244.68021,23.459450,0.995247,0.285800,382.097727,686.763592,...,0.075918,0.632293,0.056399,0.584999,0.102569,0.614978,0.082685,0.643307,0.112867,0.081959
4,1,fish_female_60-69,5.428442,8.701867,18364.95302,25.469176,1.127873,0.315651,435.578230,751.050809,...,0.080444,0.615762,0.064275,0.523287,0.106435,0.614393,0.092840,0.647070,0.128452,0.091575


In [30]:
# 按饮食类型分组并计算平均环境影响
diet_impact = calu_df.groupby('diet_group')[mean_cols + weighted_indicator_cols + ['total_env_impact']].mean().reset_index()
# 环境足迹指数(相对于最低影响的饮食类型)
min_impact = diet_impact['total_env_impact'].min()
diet_impact['env_footprint_index'] = diet_impact['total_env_impact'] / min_impact

In [31]:
# 创建一个详细的环境影响分析表
impact_analysis = []

for diet in diet_impact['diet_group']:
    diet_row = {'diet_group': diet}
    
    # 添加各环境指标的相对影响
    for base in env_indicators:
        weighted_col = f'weighted_{base}'
        mean_col = f'mean_{base}'
        
        # 找到每个指标最低的饮食类型作为基准
        min_value = diet_impact[weighted_col].min()
        
        # 计算相对影响
        rel_impact = diet_impact.loc[diet_impact['diet_group'] == diet, weighted_col].values[0] / min_value
        diet_row[f'{base}_relative_impact'] = rel_impact
        
        # 添加原始值用于参考
        diet_row[f'{base}_original'] = diet_impact.loc[diet_impact['diet_group'] == diet, mean_col].values[0]
    
    impact_analysis.append(diet_row)

impact_df = pd.DataFrame(impact_analysis)
impact_df.head()

,diet_group,ghgs_relative_impact,ghgs_original,land_relative_impact,land_original,watscar_relative_impact,watscar_original,eut_relative_impact,eut_original,ghgs_ch4_relative_impact,ghgs_ch4_original,ghgs_n2o_relative_impact,ghgs_n2o_original,bio_relative_impact,bio_original,watuse_relative_impact,watuse_original,acid_relative_impact,acid_original
0,fish,4.434060,5.452489,2.761137,7.319218,1.616916,19709.243846,3.850300,23.065932,13.783473,0.916255,2.587960,0.359165,3.238994,274.857505,2.649925,845.594953,4.343074,25.515540
1,meat,7.805220,7.937179,6.692514,13.692233,1.798993,20485.836984,6.388436,31.951860,27.462488,1.579330,5.094408,0.563690,4.421305,339.617912,3.111017,914.769192,7.224117,35.725884
2,meat100,14.702823,13.624876,15.514835,28.558030,2.649522,27061.571592,10.993493,50.675963,51.664117,2.910919,9.986961,0.998758,7.257117,507.346225,4.304949,1172.172206,12.236671,55.798517
3,meat50,4.952391,5.766550,3.634835,8.607969,1.430049,17878.297107,4.204188,23.974645,17.647777,1.083654,3.056786,0.394361,3.269930,274.592299,2.436722,782.329001,4.831925,27.016860
4,vegan,1.000000,2.551045,1.000000,4.433825,1.000000,15083.835327,1.000000,11.202768,1.000000,0.148856,1.000000,0.214003,1.000000,122.011847,1.000000,443.401156,1.000000,11.281311


### 不同饮食类型的环境影响分布

In [51]:
# 准备雷达图数据
radar_data = impact_df.copy()
relative_cols = [col for col in radar_data.columns if 'relative' in col]

# 创建雷达图
fig = go.Figure()

# 获取指标名称（简化显示）
indicators = [col.replace('_relative_impact', '') for col in relative_cols]

for i, diet in enumerate(radar_data['diet_group']):
    values = [radar_data.loc[i, f'{ind}_relative_impact'] for ind in indicators]
    # 添加第一个值以闭合图形
    values.append(values[0])
    
    fig.add_trace(go.Scatterpolar(
        r=values,
        theta=indicators + [indicators[0]],  # 闭合雷达图
        fill='toself',
        name=diet
    ))

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0.9, max(impact_df[relative_cols].max()) * 1.1]  # 设置一个合适的范围
        ),
        angularaxis=dict(
            tickfont=dict(size=12)
        )
    ),
    title="不同饮食类型在各环境指标上的相对影响",
    showlegend=True,
    legend=dict(
        title="饮食类型",
        font=dict(size=12)
    ),
    height=700,
    width=800
)

fig.write_html("diet_environmental_radar.html")
fig.show()

### 多环境指标的综合分析

In [52]:
# 准备平行坐标图数据
parallel_data = diet_impact.copy()

# 计算原始环境指标在各饮食类型间的平均值
# 选择需要展示的列
display_cols = ['diet_group', 'total_env_impact']
display_cols.extend([f'mean_{ind}' for ind in env_indicators])

# 创建平行坐标图
fig = px.parallel_coordinates(
    parallel_data,
    color="total_env_impact",
    dimensions=display_cols,
    color_continuous_scale=px.colors.diverging.Tealrose,
    color_continuous_midpoint=parallel_data['total_env_impact'].mean(),
    labels={col: col.replace('mean_', '') for col in display_cols if col.startswith('mean_')},
    title="不同饮食类型的环境影响指标比较"
)

# 布局优化
fig.update_layout(
    font=dict(size=12),
    height=600,
    width=1000,
    coloraxis_colorbar=dict(title="总环境影响")
)

fig.write_html("diet_environmental_parallel.html")
fig.show()

### 环境指标之间的关系

In [53]:
# 为散点图矩阵准备数据 - 使用环境指标均值
scatter_matrix_data = diet_impact[['diet_group'] + [f'mean_{ind}' for ind in env_indicators]]
scatter_matrix_data.columns = ['diet_group'] + env_indicators  # 简化列名

# 创建散点图矩阵
fig = px.scatter_matrix(
    scatter_matrix_data,
    dimensions=env_indicators,
    color="diet_group",
    title="环境指标之间的关系散点图矩阵",
    labels={col: col for col in env_indicators}  # 简化标签
)

# 优化布局
fig.update_traces(
    diagonal_visible=False,  # 隐藏对角线
    showupperhalf=False,    # 只显示下半部分
    marker=dict(size=6, opacity=0.7)
)

fig.update_layout(
    height=900,
    width=900,
    title_font=dict(size=16),
    font=dict(size=10)
)

fig.write_html("environmental_scatter_matrix.html")
fig.show()

### 各饮食类型的环境影响构成

In [54]:
# 准备堆叠柱状图数据
stack_data = []
for i, diet in enumerate(impact_df['diet_group']):
    for ind in indicators:
        stack_data.append({
            'diet_group': diet,
            'indicator': ind,
            'contribution': impact_df.loc[i, f'{ind}_relative_impact'] / sum(impact_df.loc[i, [f'{x}_relative_impact' for x in indicators]])
        })

stack_df = pd.DataFrame(stack_data)

# 创建堆叠柱状图
fig = px.bar(
    stack_df,
    x='diet_group',
    y='contribution',
    color='indicator',
    title='各饮食类型的环境影响构成比例',
    labels={'contribution': '贡献比例', 'diet_group': '饮食类型', 'indicator': '环境指标'},
    height=600,
    width=800
)

# 添加悬停信息
fig.update_traces(
    hovertemplate="饮食类型: %{x}<br>环境指标: %{marker.color}<br>贡献比例: %{y:.2%}<extra></extra>"
)

# 布局优化
fig.update_layout(
    yaxis=dict(title='环境影响贡献比例'),
    legend=dict(title='环境指标'),
    font=dict(size=12)
)

fig.write_html("diet_environmental_composition.html")
fig.show()

### 饮食类型的环境足迹"地图"

In [55]:
# 准备PCA输入数据
pca_input = diet_impact[[f'mean_{ind}' for ind in env_indicators]].values
pca = PCA(n_components=2)
pca_result = pca.fit_transform(pca_input)

# 创建地图数据
map_data = pd.DataFrame({
    'diet_group': diet_impact['diet_group'],
    'x': pca_result[:, 0],
    'y': pca_result[:, 1],
    'total_impact': diet_impact['total_env_impact'],
    'footprint': diet_impact['env_footprint_index']
})

# 为每个饮食类型创建"影响区域"
grid_size = 80
x = np.linspace(map_data['x'].min() - 0.5, map_data['x'].max() + 0.5, grid_size)
y = np.linspace(map_data['y'].min() - 0.5, map_data['y'].max() + 0.5, grid_size)
X, Y = np.meshgrid(x, y)
Z = np.zeros((grid_size, grid_size))

# 为每个点计算"环境影响强度"
for _, row in map_data.iterrows():
    dist = np.sqrt((X - row['x'])**2 + (Y - row['y'])**2)
    influence = row['footprint'] * np.exp(-0.8 * dist**2)
    Z += influence * 0.8

# 创建基础图
fig = go.Figure()

# 添加环境影响"地图"
fig.add_trace(go.Contour(
    z=Z,
    x=x,
    y=y,
    colorscale="YlOrRd",
    contours=dict(
        start=0, 
        end=Z.max(), 
        size=(Z.max()-0)/15,
        showlabels=False  # 不显示等高线标签，减少混乱
    ),
    colorbar=dict(
        title="环境影响强度",
        titleside="right",
        titlefont=dict(size=12)
    ),
    line=dict(width=0.5, color="rgba(150, 150, 150, 0.5)"),
    showscale=True,
    opacity=0.5  # 更透明
))

# 添加饮食类型位置点和标签
for i, row in map_data.iterrows():
    diet = row['diet_group']
    
    # 添加点
    fig.add_trace(go.Scatter(
        x=[row['x']],
        y=[row['y']],
        mode="markers",
        marker=dict(
            size=15,
            color="black",
            symbol="circle-open",
            line=dict(width=3)
        ),
        name=diet,
        showlegend=False,
        hoverinfo="text",
        hovertext=f"<b>{diet}</b><br>环境足迹指数: {row['footprint']:.2f}"
    ))
    
    # 为每个饮食直接使用单独的标签（不用引导线）
    # 这样可以更灵活地放置标签，避免遮挡
    if diet == 'meat':
        # 将meat标签放在点的右侧
        fig.add_annotation(
            x=row['x'],
            y=row['y'],
            text=f"{diet}<br>{row['footprint']:.2f}",
            showarrow=False,
            xshift=45,  # 向右移动标签
            yshift=5,   # 略微向上调整
            font=dict(size=12, color="black"),
            bgcolor="rgba(255, 255, 255, 0.8)",  # 半透明白色背景
            bordercolor="black",
            borderwidth=1,
            borderpad=3
        )
    elif diet == 'meat50':
        # 将meat50标签放在点的左上方
        fig.add_annotation(
            x=row['x'],
            y=row['y'],
            text=f"{diet}<br>{row['footprint']:.2f}",
            showarrow=False,
            xshift=-50,  # 向左移动标签
            yshift=25,   # 向上调整
            font=dict(size=12, color="black"),
            bgcolor="rgba(255, 255, 255, 0.8)",
            bordercolor="black",
            borderwidth=1,
            borderpad=3
        )
    elif diet == 'fish':
        # 将fish标签放在点的右下方
        fig.add_annotation(
            x=row['x'],
            y=row['y'],
            text=f"{diet}<br>{row['footprint']:.2f}",
            showarrow=False,
            xshift=10,   # 略微向右
            yshift=-45,  # 向下调整
            font=dict(size=12, color="black"),
            bgcolor="rgba(255, 255, 255, 0.8)",
            bordercolor="black",
            borderwidth=1,
            borderpad=3
        )
    elif diet == 'meat100':
        # 将meat100标签放在点的左下方
        fig.add_annotation(
            x=row['x'],
            y=row['y'],
            text=f"{diet}<br>{row['footprint']:.2f}",
            showarrow=False,
            xshift=-50,  # 向左移动标签
            yshift=-30,  # 向下调整
            font=dict(size=12, color="black"),
            bgcolor="rgba(255, 255, 255, 0.8)",
            bordercolor="black",
            borderwidth=1,
            borderpad=3
        )
    elif diet == 'vegan':
        # 将vegan标签放在点的右上方
        fig.add_annotation(
            x=row['x'],
            y=row['y'],
            text=f"{diet}<br>{row['footprint']:.2f}",
            showarrow=False,
            xshift=50,   # 向右移动标签
            yshift=30,   # 向上调整
            font=dict(size=12, color="black"),
            bgcolor="rgba(255, 255, 255, 0.8)",
            bordercolor="black",
            borderwidth=1,
            borderpad=3
        )
    elif diet == 'veggie':
        # 将veggie标签放在点的左上方
        fig.add_annotation(
            x=row['x'],
            y=row['y'],
            text=f"{diet}<br>{row['footprint']:.2f}",
            showarrow=False,
            xshift=-50,  # 向左移动标签
            yshift=50,   # 向上调整
            font=dict(size=12, color="black"),
            bgcolor="rgba(255, 255, 255, 0.8)",
            bordercolor="black", 
            borderwidth=1,
            borderpad=3
        )

# 优化布局
fig.update_layout(
    title=dict(
        text="饮食类型的'环境足迹地图'<br><sup>基于主成分分析映射环境影响到二维空间</sup>",
        font=dict(size=16)
    ),
    xaxis=dict(
        title="环境影响主成分1 (99.96%)",
        showgrid=False,
        zeroline=False,
        showticklabels=False
    ),
    yaxis=dict(
        title="环境影响主成分2 (0.04%)",
        showgrid=False,
        zeroline=False,
        showticklabels=False
    ),
    height=700,
    width=800,
    plot_bgcolor="rgba(240, 240, 240, 0.8)"
)

# 添加PCA解释方差标签
explained_variance = pca.explained_variance_ratio_
fig.add_annotation(
    x=0.98,
    y=0.02,
    xref="paper",
    yref="paper",
    text=f"<b>主成分解释方差:</b><br>主成分1: {explained_variance[0]:.2%}<br>主成分2: {explained_variance[1]:.2%}",
    showarrow=False,
    font=dict(size=12),
    bgcolor="white",
    bordercolor="black",
    borderwidth=1,
    borderpad=4,
    align="right"
)

fig.write_html("final_environmental_impact_map.html")
fig.show()

In [ ]:
html_code = """
<!DOCTYPE html>
<html>
<head>
    <title>饮食类型环境影响分析仪表板</title>
    <meta charset="UTF-8">
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
            background-color: #f5f5f5;
            color: #333;
        }
        .dashboard-container {
            max-width: 1200px;
            margin: 0 auto;
            padding: 20px;
        }
        h1 {
            text-align: center;
            color: #2c3e50;
            margin-bottom: 30px;
            padding-bottom: 15px;
            border-bottom: 2px solid #3498db;
        }
        .slide-container {
            background: white;
            padding: 20px;
            border-radius: 8px;
            box-shadow: 0 2px 10px rgba(0,0,0,0.1);
            margin-bottom: 20px;
            display: none;
            transition: opacity 0.3s ease;
        }
        .slide-container.active {
            display: block;
        }
        .nav-buttons {
            text-align: center;
            margin: 20px 0;
        }
        button {
            background: #3498db;
            color: white;
            border: none;
            padding: 12px 24px;
            margin: 0 10px;
            border-radius: 30px;
            cursor: pointer;
            font-size: 16px;
            transition: background 0.3s ease;
        }
        button:hover {
            background: #2980b9;
        }
        .slide-dots {
            text-align: center;
            margin: 20px 0;
        }
        .dot {
            height: 12px;
            width: 12px;
            margin: 0 8px;
            background-color: #bbb;
            border-radius: 50%;
            display: inline-block;
            cursor: pointer;
            transition: background-color 0.3s ease;
        }
        .dot.active {
            background-color: #3498db;
        }
        .iframe-container {
            width: 100%;
            height: 700px;
            border-radius: 4px;
            overflow: hidden;
        }
        iframe {
            width: 100%;
            height: 100%;
            border: none;
        }
        .slide-title {
            margin-top: 0;
            color: #2c3e50;
            padding-bottom: 10px;
            font-size: 24px;
        }
        .slide-description {
            color: #7f8c8d;
            margin-bottom: 20px;
            font-size: 16px;
            line-height: 1.5;
        }
        .navigation-help {
            text-align: center;
            font-size: 14px;
            color: #7f8c8d;
            margin-top: 20px;
        }
        .footer {
            text-align: center;
            margin-top: 40px;
            padding-top: 20px;
            border-top: 1px solid #ddd;
            color: #7f8c8d;
            font-size: 14px;
        }
    </style>
</head>
<body>
    <div class="dashboard-container">
        <h1>饮食类型的环境影响分析</h1>
        
        <div class="nav-buttons">
            <button onclick="prevSlide()">上一个</button>
            <button onclick="nextSlide()">下一个</button>
        </div>
        
        <div class="slide-dots" id="slide-dots">
            <!-- 将由JavaScript填充 -->
        </div>
        
        <div class="slides" id="slides">
            <!-- 幻灯片内容 -->
            <div class="slide-container">
                <h2 class="slide-title">1. 饮食类型的环境影响构成比例</h2>
                <p class="slide-description">此图展示了不同饮食类型对各种环境指标的影响构成比例，帮助理解每种饮食在不同环境方面的相对影响。</p>
                <div class="iframe-container">
                    <iframe src="diet_environmental_composition.html"></iframe>
                </div>
            </div>
            
            <div class="slide-container">
                <h2 class="slide-title">2. 各饮食类型的环境影响多维对比</h2>
                <p class="slide-description">平行坐标图展示了各个饮食类型在所有环境指标上的表现，便于多维度比较不同饮食类型的环境影响。</p>
                <div class="iframe-container">
                    <iframe src="diet_environmental_parallel.html"></iframe>
                </div>
            </div>
            
            <div class="slide-container">
                <h2 class="slide-title">3. 不同饮食类型的环境影响特征分布</h2>
                <p class="slide-description">雷达图直观地展示了各饮食类型在不同环境指标上的相对影响，展现每种饮食的环境影响特征。</p>
                <div class="iframe-container">
                    <iframe src="diet_environmental_radar.html"></iframe>
                </div>
            </div>
            
            <div class="slide-container">
                <h2 class="slide-title">4. 环境指标之间的关系散点图矩阵</h2>
                <p class="slide-description">散点图矩阵展示了各环境指标之间的相互关系，帮助发现环境指标间的相关性和模式。</p>
                <div class="iframe-container">
                    <iframe src="environmental_scatter_matrix.html"></iframe>
                </div>
            </div>
            
            <div class="slide-container">
                <h2 class="slide-title">5. 饮食类型的'环境足迹地图'</h2>
                <p class="slide-description">这张创意地图将环境影响数据通过主成分分析映射到二维空间，直观展示不同饮食类型的环境足迹分布。</p>
                <div class="iframe-container">
                    <iframe src="final_environmental_impact_map.html"></iframe>
                </div>
            </div>
        </div>
        
        <div class="navigation-help">
            提示：您可以使用键盘左右箭头键进行导航，也可以在每个图表中使用鼠标交互（悬停、缩放、平移等）
        </div>
        
        <div class="footer">
            © 2025 饮食与环境影响分析 | 数据来源：牛津大学研究团队
        </div>
    </div>
    
    <script>
        let currentSlide = 0;
        const slides = document.querySelectorAll('.slide-container');
        const totalSlides = slides.length;
        
        // 初始化点导航
        const dotsContainer = document.getElementById('slide-dots');
        for (let i = 0; i < totalSlides; i++) {
            const dot = document.createElement('span');
            dot.classList.add('dot');
            dot.onclick = function() { goToSlide(i); };
            dotsContainer.appendChild(dot);
        }
        
        // 显示第一张幻灯片
        function showSlides() {
            for (let i = 0; i < totalSlides; i++) {
                slides[i].classList.remove('active');
                document.querySelectorAll('.dot')[i].classList.remove('active');
            }
            slides[currentSlide].classList.add('active');
            document.querySelectorAll('.dot')[currentSlide].classList.add('active');
        }
        
        // 下一张幻灯片
        function nextSlide() {
            currentSlide = (currentSlide + 1) % totalSlides;
            showSlides();
        }
        
        // 上一张幻灯片
        function prevSlide() {
            currentSlide = (currentSlide - 1 + totalSlides) % totalSlides;
            showSlides();
        }
        
        // 转到特定幻灯片
        function goToSlide(n) {
            currentSlide = n;
            showSlides();
        }
        
        // 初始化
        showSlides();
        
        // 键盘导航
        document.addEventListener('keydown', function(e) {
            if (e.key === 'ArrowRight') {
                nextSlide();
            } else if (e.key === 'ArrowLeft') {
                prevSlide();
            }
        });
    </script>
</body>
</html>
"""

# 将HTML代码保存到文件
with open('diet_environmental_dashboard.html', 'w', encoding='utf-8') as f:
    f.write(html_code)

print("HTML仪表板文件已成功创建！")